In [1]:
import pandas
import nipy
from nipy.modalities.fmri import hrf

In [2]:
df = pandas.read_csv('data/sim-data_drift-SP-variability-estim-onep.csv', index_col=0)

In [3]:
import numpy as np

In [4]:
df['onset'] = np.arange(10, len(df)*10 + 1, 10)

In [5]:
def get_bold_response(df, method='time_on_task',
                      sample_rate=1000):
    
    neural_signal = np.zeros((df.onset.max() + 20) * sample_rate)
    
    hrf = nipy.modalities.fmri.hrf.spm_hrf_compat(np.arange(0, 20, 1./sample_rate))
    
    for ix, row in df.iterrows():
        
        if method == 'time_on_task':
            start_idx = int(row.onset * sample_rate)
            end_idx = int((row.onset + row.rt)*sample_rate)
            value = 1
        
        neural_signal[start_idx:end_idx] = value
    
    bold_signal = np.convolve(hrf, neural_signal)
    
    return neural_signal, bold_signal
    
    

# Time on task

In [ ]:


import matplotlib.pyplot as plt
%matplotlib inline

In [41]:
sample_rate = 100

In [42]:
neural_signal, bold_signal = get_bold_response(df, sample_rate=sample_rate)

In [43]:
from nipy.modalities.fmri.experimental_paradigm import EventRelatedParadigm

In [44]:
paradigm = EventRelatedParadigm(['trial %d' % (ix+1) for ix in df.index], df.onset.tolist())

In [45]:
from scipy import signal

In [46]:
tr = 2.0

In [47]:
len(bold_signal) / sample_rate * tr

40079.98

In [48]:
bold_signal = signal.resample_poly(bold_signal, 1, sample_rate * tr)

In [49]:
import nipy.modalities.fmri.design_matrix as dm


In [87]:
n_scans = len(bold_signal)

# paradigm
frametimes = np.linspace(0, (n_scans - 1) * tr, n_scans)

# confounds
hrf_model = 'canonical'
drift_model = "polynomial"

X, names = dm.dmtx_light(frametimes, paradigm, drift_model=drift_model, drift_order=0, hrf_model=hrf_model)

In [88]:
import statsmodels.api as sm

In [89]:
model = sm.OLS(bold_signal, pandas.DataFrame(X, columns=names))

In [90]:
results = model.fit()

In [110]:
single_trial_betas = pandas.DataFrame(results.params).reset_index()
single_trial_betas = single_trial_betas.drop(2000)
single_trial_betas = single_trial_betas.rename(columns={0:'beta'})
single_trial_betas['trial'] = single_trial_betas['index'].apply(lambda x: int(x.split()[-1]))

In [111]:
df.merge(single_trial_betas, on='trial')

,SP_m,SP_sd,SP_t,accuracy,drift_m,drift_sd,drift_t,ndt,participant,rt,threshold,trial,onset,index,beta
0,-0.1,0.1,0.474455,1,2,1.2,1.971660,0.3,1,0.525,1.2,1,10,trial 1,2.591663
1,-0.1,0.1,0.421582,1,2,1.2,2.412196,0.3,1,0.521,1.2,2,20,trial 2,2.711804
2,-0.1,0.1,0.440954,1,2,1.2,1.552231,0.3,1,0.641,1.2,3,30,trial 3,3.528447
3,-0.1,0.1,0.434787,1,2,1.2,2.604858,0.3,1,0.553,1.2,4,40,trial 4,3.016930
4,-0.1,0.1,0.365565,1,2,1.2,1.242914,0.3,1,0.442,1.2,5,50,trial 5,2.268155
5,-0.1,0.1,0.478541,1,2,1.2,0.895003,0.3,1,0.542,1.2,6,60,trial 6,2.851792
6,-0.1,0.1,0.466241,1,2,1.2,0.981826,0.3,1,1.065,1.2,7,70,trial 7,6.247759
7,-0.1,0.1,0.532411,1,2,1.2,2.115955,0.3,1,0.436,1.2,8,80,trial 8,2.454801
8,-0.1,0.1,0.422569,1,2,1.2,2.889608,0.3,1,0.702,1.2,9,90,trial 9,3.948225
9,-0.1,0.1,0.513589,1,2,1.2,3.431862,0.3,1,0.560,1.2,10,100,trial 10,3.121037


In [112]:
df.to_csv('data/sim-data_drift-SP-variability-estim-onep.csv')

0          1
1         10
2        100
3       1000
4       1001
5       1002
6       1003
7       1004
8       1005
9       1006
10      1007
11      1008
12      1009
13       101
14      1010
15      1011
16      1012
17      1013
18      1014
19      1015
20      1016
21      1017
22      1018
23      1019
24       102
25      1020
26      1021
27      1022
28      1023
29      1024
        ... 
1970     972
1971     973
1972     974
1973     975
1974     976
1975     977
1976     978
1977     979
1978      98
1979     980
1980     981
1981     982
1982     983
1983     984
1984     985
1985     986
1986     987
1987     988
1988     989
1989      99
1990     990
1991     991
1992     992
1993     993
1994     994
1995     995
1996     996
1997     997
1998     998
1999     999
Name: trial, Length: 2000, dtype: int64